## Task 4.9-Part 1

#### Importing libraries

In [1]:
import pandas as pd
import numpy as np
import os
import gc

In [2]:
# importing customers file
path = r'C:\Users\Ali\OneDrive\Desktop\careerfoundry\Data Immersion\Achievement 4\15-08-2025 Instacart Basket Analysis'
df_customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'prepared Data', 'orders_products_merged_grouped.pkl'))

In [3]:
df_customers.shape

(206209, 10)

In [4]:
df_customers.head(10)

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374
5,133128,Cynthia,Noble,Female,Kentucky,43,1/1/2017,2,married,49643
6,152052,Chris,Walton,Male,Montana,20,1/1/2017,0,single,61746
7,168851,Joseph,Hickman,Male,South Carolina,30,1/1/2017,0,single,63712
8,69965,Jeremy,Vang,Male,Texas,47,1/1/2017,1,married,162432
9,82820,Shawn,Chung,Male,Virginia,26,1/1/2017,2,married,32072


In [5]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


In [6]:
df_customers.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


#### Missing values check

In [7]:
 # True if any missing values in whole DataFrame
print(df_customers.isnull().values.any())

True


In [8]:
# Total number of missing values
print(df_customers.isnull().values.sum()) 

11259


In [9]:
print(df_customers.isnull().sum())

user_id             0
First Name      11259
Surnam              0
Gender              0
STATE               0
Age                 0
date_joined         0
n_dependants        0
fam_status          0
income              0
dtype: int64


##### Customer's first name is not a critical information for our analysis so we will not delete the rows with missing first names and will continue working with our original data frame.

#### Renaming column Names

In [10]:
df_customers.rename(columns = {'Surnam' : 'last_name'}, inplace = True)

In [11]:
df_customers.rename(columns = {'STATE' : 'state'}, inplace = True)

In [12]:
df_customers.rename(columns = {'fam_status' : 'family_status'}, inplace = True)

In [13]:
df_customers.rename(columns = {'First Name' : 'first_name'}, inplace = True)

In [14]:
df_customers.rename(columns = {'Gender' : 'gender'}, inplace = True)

In [15]:
df_customers.rename(columns = {'Age' : 'age'}, inplace = True)

In [16]:
df_customers.head(10)

,user_id,first_name,last_name,gender,state,age,date_joined,n_dependants,family_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374
5,133128,Cynthia,Noble,Female,Kentucky,43,1/1/2017,2,married,49643
6,152052,Chris,Walton,Male,Montana,20,1/1/2017,0,single,61746
7,168851,Joseph,Hickman,Male,South Carolina,30,1/1/2017,0,single,63712
8,69965,Jeremy,Vang,Male,Texas,47,1/1/2017,1,married,162432
9,82820,Shawn,Chung,Male,Virginia,26,1/1/2017,2,married,32072


#### Checking for mixed data types

In [17]:
# function to check for mixed data types
for col in df_customers.columns.tolist():
  weird = (df_customers[[col]].map(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customers[weird]) > 0:
    print (col)

first_name


In [18]:
# changing data type to string keeping the NaN values(not converting NaN to string)
df_customers['first_name'] = df_customers['first_name'].astype('string')

In [19]:
df_customers['first_name'].dtype

string[python]

#### Handling Duplicates

In [20]:
df_dups_customers = df_customers[df_customers.duplicated()]

In [21]:
df_dups_customers

,user_id,first_name,last_name,gender,state,age,date_joined,n_dependants,family_status,income


##### There are no duplicates in customers data frame.

#### Exporting clean Data frame

In [22]:
df_customers.to_csv(os.path.join(path, '02 Data','Prepared Data', 'customers_checked.csv'), index = False)

#### Merging Data Frames

In [23]:
df_customers.shape

(206209, 10)

In [24]:
df_customers.head()

,user_id,first_name,last_name,gender,state,age,date_joined,n_dependants,family_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [25]:
ords_prods_merge.shape

(32404289, 23)

In [26]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,is_first_order,product_id,add_to_cart_order,reordered,...,prices,price_range_loc,busiest_days,busiest_period_of_day,max_order,loyalty_flag,Average_price,median_days_between_orders,spending_flag,order_frequency_flag
0,2539329,1,1,2,8,NaN,1,196,1,0,...,9.0,Mid-range product,Regularly busy,Average Orders,10,New customer,6.367797,20.5,Low spender,Non-frequent customer
1,2539329,1,1,2,8,NaN,1,14084,2,0,...,12.5,Mid-range product,Regularly busy,Average Orders,10,New customer,6.367797,20.5,Low spender,Non-frequent customer
2,2539329,1,1,2,8,NaN,1,12427,3,0,...,4.4,Low-range product,Regularly busy,Average Orders,10,New customer,6.367797,20.5,Low spender,Non-frequent customer
3,2539329,1,1,2,8,NaN,1,26088,4,0,...,4.7,Low-range product,Regularly busy,Average Orders,10,New customer,6.367797,20.5,Low spender,Non-frequent customer
4,2539329,1,1,2,8,NaN,1,26405,5,0,...,1.0,Low-range product,Regularly busy,Average Orders,10,New customer,6.367797,20.5,Low spender,Non-frequent customer


In [27]:
ords_prods_merge.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32404289 entries, 0 to 32434488
Data columns (total 23 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   order_id                    int64  
 1   user_id                     int64  
 2   order_number                int64  
 3   orders_day_of_week          int64  
 4   order_hour_of_day           int64  
 5   days_since_prior_order      float64
 6   is_first_order              int64  
 7   product_id                  int64  
 8   add_to_cart_order           int64  
 9   reordered                   int64  
 10  product_name                object 
 11  aisle_id                    Int64  
 12  department_id               Int64  
 13  prices                      float64
 14  price_range_loc             object 
 15  busiest_days                object 
 16  busiest_period_of_day       object 
 17  max_order                   int64  
 18  loyalty_flag                object 
 19  Average_price           

In [28]:
# merging both data frames
df_merged_final = ords_prods_merge.merge(df_customers, on = 'user_id', indicator = True)

In [29]:
df_merged_final['_merge'].value_counts()

_merge
both          32404289
left_only            0
right_only           0
Name: count, dtype: int64

In [30]:
df_merged_final.shape

(32404289, 33)

In [31]:
# exporting the merged data frame
df_merged_final.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'final_dataframe.pkl'))

In [32]:
# checking our merge
df_merged_final_check = ords_prods_merge.merge(df_customers, how = 'outer', on = 'user_id', indicator = True)

In [33]:
df_merged_final_check['_merge'].value_counts()

_merge
both          32404289
left_only            0
right_only           0
Name: count, dtype: int64

#### We have a full merge.